In [ ]:
import requests
import numpy as np
import pandas as pd
from mpl_toolkits.basemap import Basemap
import matplotlib.cm as cm
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize, LinearSegmentedColormap
import matplotlib.colors as colors
from matplotlib.mlab import griddata
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.interpolate import griddata as gd
from shapely.geometry import mapping, Point, Polygon, LineString, MultiPolygon, MultiPoint, box
from shapely.ops import unary_union
from descartes import PolygonPatch
import math
import fiona
from itertools import chain, permutations, combinations
import json
import networkx as nx
from lxml import etree
%matplotlib inline
mpl.rcParams['figure.figsize'] = (16, 12)
mpl.rcParams['savefig.dpi'] = 200
mpl.rcParams['axes.grid'] = False   # display grid or not

In [ ]:
from matplotlib import rc
rc('font', **{'family':'sans-serif',
    'sans-serif':['Helvetica'],
    'monospace': ['Inconsolata'],
    'serif': ['Helvetica']})
rc('text', **{'usetex': True})
rc('text', **{'latex.preamble': '\usepackage{sfmath}'})

In [ ]:
shp = fiona.open('data/london_wards.shp')
crs_data = shp.crs
bds = shp.bounds
shp.close()
extra = 0.01
ll = (bds[0], bds[1])
ur = (bds[2], bds[3])
coords = list(chain(ll, ur))
w, h = coords[2] - coords[0], coords[3] - coords[1]

In [ ]:
m = Basemap(
    projection='tmerc',
    lon_0 = -2.,
    lat_0 = 49.,
    ellps = 'WGS84',
    llcrnrlon=coords[0] - extra * w,
    llcrnrlat=coords[1] - extra + 0.01 * h,
    urcrnrlon=coords[2] + extra * w,
    urcrnrlat=coords[3] + extra + 0.01 * h,
    lat_ts=0,
    resolution='i',
    suppress_ticks=True)
m.readshapefile(
    'data/london_wards',
    'london',
    color='none',
    zorder=2)

In [ ]:
# cascade ward polygons together
london_poly = unary_union([Polygon(xy) for xy in m.london])

In [ ]:
def query_travel_time(start, end, method):
    """
    Get a travel time back from MapZen's OSRM
    start, end: lon, lat tuples
    method: foot, car, bicycle
    returns travel time, in seconds
    TODO: bounds checking for coords
    """
    allowed = ('foot', 'car', 'bicycle')
    if method not in allowed:
        raise Exception(
            "Unknown method. Must be one of %s. Christ." % ', '.join(allowed))
    endpoint = 'http://osrm.mapzen.com'
    method = '/{m}/viaroute'.format(m=method)
    # should be properly encoding second loc, but dict keys are unique!
    # reverse lon, lat because ugh    
    params = {'loc': '{1},{0}&loc={3},{2}'.format(*chain(start, end))}
    req = requests.get(endpoint + method, params=params)
    try:
        req.raise_for_status()
    except requests.exceptions.HTTPError:
        return np.nan
    if req.json()['status'] == 207:
        return np.nan
    return req.json()['route_summary']['total_time']

In [ ]:
def project_linestring(ls, inverse=False):
    """ return a linestring projected into map coordinates """
    if not pd.isnull(ls):
        return LineString(zip(*m(*zip(*ls.coords))))
    else:
        return np.nan

In [ ]:
# parse XML into dict
tree = etree.parse("data/bike_stations.xml")
root = tree.getroot()

output = dict()
output['name'] = []
output['lon'] = []
output['lat'] = []

for each in root.xpath('station'):
    output['name'].append(each[1].text)
    output['lon'].append(each[4].text)
    output['lat'].append(each[3].text)

stations = pd.DataFrame(output)
stations[['lon', 'lat']] = stations[['lon', 'lat']].astype(float)

In [ ]:
# work out station centroid and WGS84 lon / lat
centroid = zip(*m(*zip(*MultiPoint(stations[['lon', 'lat']].values).convex_hull.centroid.coords)))[0]
centroid

In [ ]:
# move it onto Westminster Bridge
centroid = m(-0.12204, 51.50083)
centroid, m(*centroid, inverse=True)

In [ ]:
# project lon / lat coords
stations['projected_lon'], stations['projected_lat'] = m(*(stations["lon"].values, stations["lat"].values))

In [ ]:
# calculate station distance from centroid
stations['centroid_distance'] = stations.apply(lambda x:
    math.sqrt(
        ((abs(centroid[0] - x['projected_lon']) ** 2) + (abs(centroid[1] - x['projected_lat']) ** 2))
    ), axis=1)

You can skip the next cell and import travel times from CSV if you wish (don't divide travel by 60, though):  
`stations = pd.read_csv('data/stations_travel_time.csv', index_col=0)`

In [ ]:
def travel_time(df, start):
    """ return travel times between a given centroid and all stations in the network """
    return query_travel_time(start, (df['lon'], df['lat']), 'bicycle')


stations['travel_time'] = stations.apply(travel_time, args=(m(*centroid, inverse=True),), axis=1)

In [ ]:
# travel time in minutes is more useful
stations['travel_time'] /=  60.
# replace travel time of < 1 with 1. minutes - not sure about this
stations.loc[stations['travel_time'] <= 1., 'travel_time'] = 1.
# save our original DataFrame - we're going to drop empty travel time stations
stations_orig = pd.DataFrame(stations)
stations = stations.dropna()

In [ ]:
nodes = pd.read_csv('data/nodes_london.csv', header=None, names=['name', 'lat', 'lon'])
nodes.head()

In [ ]:
nodes['coords'] = zip(nodes['lon'], nodes['lat'])
nodes = nodes.set_index('name')
nodes.head()

In [ ]:
nodes = nodes[:20]
nodes

In [ ]:
traveltime_square = pd.DataFrame(
    np.zeros(len(nodes) ** 2).reshape(len(nodes), len(nodes)),
    index=nodes.index, columns=nodes.index)
traveltime_square.head()

In [ ]:
def get_travel_time(col):
    """
    retrieve coords from nodes df
    pass to query_routing_machine()
    """
    end = nodes.ix[col.name]['coords']
    ns = nodes['coords'].apply(query_travel_time, end=end, method='bicycle')
    return ns

In [ ]:
routes_travel_time = traveltime_square.apply(get_travel_time, axis=1).T

In [ ]:
routes_travel_time.head()

In [ ]:
def compute_travel_times(df, src):
    """
    Return new DataFrame based on simple network analysis of origin and destination
    We could calculate this outselves, but it's good to have NetworkX for future
    work
    """
    travel_time = nx.from_numpy_matrix(df.fillna(0).values)
    travel_time = nx.relabel_nodes(travel_times, dict(enumerate(routes_travel_time.columns)))
    ttimes = nx.single_source_dijkstra_path_length(
        travel_time, source=src, cutoff=None, weight='weight')
    df_ttimes = pd.DataFrame({'destination': ttimes.keys(), 'travel_time': ttimes.values()})
    return df_ttimes.sort('destination')

In [ ]:
df_ttimes = compute_travel_times(routes_travel_time, 'Abbey Orchard Street')

In [ ]:
df_ttimes

In [ ]:
df_ttimes['projected_lon'], df_ttimes['projected_lat'] = m(*(nodes["lon"].values, nodes["lat"].values))
df_ttimes

In [ ]:
# set up a square grid with the same extents as our measured data
numcols, numrows = 1000, 1000
xi = np.linspace(df_ttimes['projected_lon'].min(), df_ttimes['projected_lon'].max(), numcols)
yi = np.linspace(df_ttimes['projected_lat'].min(), df_ttimes['projected_lat'].max(), numrows)
# get lon and lat coords of our grid points
xi, yi = np.meshgrid(xi, yi)
# interpolate
zi = gd(
    (df_ttimes[['projected_lon', 'projected_lat']]),
    df_ttimes['travel_time'].values,
    (xi, yi),
    method='linear')

In [ ]:
norm = Normalize()

plt.clf()
fig = plt.figure()
ax = fig.add_subplot(111, axisbg='w', frame_on=False)

m.drawparallels(
    np.arange(ll[0], ur[0], 2.),
    color = 'black', linewidth = 0.5,
    labels=[True, False, False, False], ax=ax)
m.drawmeridians(
    np.arange(ll[1], ur[1], 2.),
    color = 'black', linewidth = 0.5,
    labels=[False, False, False, True], ax=ax)

# plot London outline
lp = PolygonPatch(
    london_poly,
    fc='#555555', ec='#787878', lw=.25, alpha=.5, zorder=3)
ax.add_patch(lp)

# contour plots
# con = m.contour(xi, yi, zi, 15, zorder=4, linewidths=.25, linestyles='dashed', colors='k', alpha=0.6, ax=ax)
conf = m.contourf(xi, yi, zi, 15, zorder=4, alpha=0.85, cmap='coolwarm', ax=ax)

# scatter plot - vmin/max for colormap compat
m.scatter(
    df_ttimes['projected_lon'],
    df_ttimes['projected_lat'],
    color='#545454',
    edgecolor='#ffffff',
    alpha=.85,
    s=30 * norm(df_ttimes['travel_time']),
    cmap='RdBu',
    ax=ax,
    vmin=zi.min(), vmax=zi.max(), zorder=5)

# add origin, assumed to be first row in df
m.scatter(
    df_ttimes.iloc[0]['projected_lon'],
    df_ttimes.iloc[0]['projected_lat'],
    s=20, edgecolor='#000000', alpha=1.,
    color='w', zorder=5)

# add colour bar, title, and scale
cbar = plt.colorbar(conf, shrink=.5)
cbar.set_label("Travel Time - Seconds", size=8)

# We need to reproduce the OS Data copyright licence
smallprint = ax.text(
    1.03, 0,
    'Contains Ordnance Survey data\n$\copyright$ Crown copyright and database right 2014\n\nDarker routes are longer',
    ha='right', va='bottom',
    size=6,
    color='#555555',
    transform=ax.transAxes,
)

plt.title(
    "%s-Point Grid Natural Neighbour Delaunay Interpolation" % numcols, fontsize=10)
m.drawmapscale(
    coords[0] + 0.08, coords[1] + 0.015,
    coords[0], coords[1],
    10.,
    barstyle='fancy', labelstyle='simple',
    fillcolor1='w', fillcolor2='#555555',
    fontcolor='#555555',
    zorder=5)

plt.tight_layout()
# plt.savefig("data/isochrone.png", format="png", bbox_inches='tight', transparent=True, dpi=300)
plt.show()